In [ ]:
import math
import os.path
os.environ["PYOPENGL_PLATFORM"] = "egl"

import matplotlib.pyplot as plt
import numpy as np
import pyrender
import trimesh
import json
from tqdm import tqdm


In [ ]:
color2label = {
    # upper label 1-8 UL1-8, label 9-16 UR1-8
    (170, 255, 127): ("aaff7f", "UL1", 1),
    (170, 255, 255): ("aaffff", "UL2", 2),
    (255, 255, 0): ("ffff00", "UL3", 3),
    (255, 170, 0): ("ffaa00", "UL4", 4),
    (170, 170, 255): ("aaaaff", "UL5", 5),
    (0, 170, 255): ("00aaff", "UL6", 6),
    (85, 170, 0): ("55aa00", "UL7", 7),
    (204, 204, 15): ("cccc0f", "UL8", 8),

    (255, 85, 255): ("ff55ff", "UR1", 9),
    (255, 85, 127): ("ff557f", "UR2", 10),
    (85, 170, 127): ("55aa7f", "UR3", 11),
    (255, 85, 0): ("ff5500", "UR4", 12),
    (0, 85, 255): ("0055ff", "UR5", 13),
    (170, 0, 0): ("aa0000", "UR6", 14),
    (73, 247, 235): ("49f7eb", "UR7", 15),
    (125, 18, 247): ("7d12f7", "UR8", 16),

    # lower 1-8 LL1-8, 9-16 LR1-8
    (240, 0, 0): ("f00000", "LL1", 1),
    (251, 255, 3): ("fbff03", "LL2", 2),
    (44, 251, 255): ("2cfbff", "LL3", 3),
    (241, 47, 255): ("f12fff", "LL4", 4),
    (125, 255, 155): ("7dff9b", "LL5", 5),
    (26, 125, 255): ("1a7dff", "LL6", 6),
    (255, 234, 157): ("ffea9d", "LL7", 7),
    (204, 126, 126): ("cc7e7e", "LL8", 8),

    (206, 129, 212): ("ce81d4", "LR1", 9),
    (45, 135, 66): ("2d8742", "LR2", 10),
    (185, 207, 45): ("b9cf2d", "LR3", 11),
    (69, 147, 207): ("4593cf", "LR4", 12),
    (207, 72, 104): ("cf4868", "LR5", 13),
    (4, 207, 4): ("04cf04", "LR6", 14),
    (35, 1, 207): ("2301cf", "LR7", 15),
    (82, 204, 169): ("52cca9", "LR8", 16),

    # gum
    (125, 125, 125): ("7d7d7d", 'GUM', 0),
}

In [ ]:
label2color_lower = {
    1: ("f00000", "LL1", (240, 0, 0)),
    2: ("fbff03", "LL2", (251, 255, 3)),
    3: ("2cfbff", "LL3", (44, 251, 255)),
    4: ("f12fff", "LL4", (241, 47, 255)),
    5: ("7dff9b", "LL5", (125, 255, 155)),
    6: ("1a7dff", "LL6", (26, 125, 255)),
    7: ("ffea9d", "LL7", (255, 234, 157)),
    8: ("cc7e7e", "LL8", (204, 126, 126)),

    9: ("ce81d4", "LR1", (206, 129, 212)),
    10: ("2d8742", "LR2", (45, 135, 66)),
    11: ("b9cf2d", "LR3", (185, 207, 45)),
    12: ("4593cf", "LR4", (69, 147, 207)),
    13: ("cf4868", "LR5", (207, 72, 104)),
    14: ("04cf04", "LR6", (4, 207, 4)),
    15: ("2301cf", "LR7", (35, 1, 207)),
    16: ("52cca9", "LR8", (82, 204, 169)),

    # gum
    0: ("7d7d7d", 'GUM', (125, 125, 125)),
}

label2color_upper = {
    1: ("aaff7f", "UL1", (170, 255, 127)),
    2: ("aaffff", "UL2", (170, 255, 255)),
    3: ("ffff00", "UL3", (255, 255, 0)),
    4: ("faa00", "UL4", (255, 170, 0)),
    5: ("aaaaff", "UL5", (170, 170, 255)),
    6: ("00aaff", "UL6", (0, 170, 255)),
    7: ("55aa00", "UL7", (85, 170, 0)),
    8: ("cccc0f", "UL8", (204, 204, 15)),

    9: ("ff55ff", "UR1", (255, 85, 255)),
    10: ("ff557f", "UR2", (255, 85, 127)),
    11: ("55aa7f", "UR3", (85, 170, 127)),
    12: ("ff5500", "UR4", (255, 85, 0)),
    13: ("0055ff", "UR5", (0, 85, 255)),
    14: ("aa0000", "UR6", (170, 0, 0)),
    15: ("49f7eb", "UR7", (73, 247, 235)),
    16: ("7d12f7", "UR8", (125, 18, 247)),

    # gum
    0: ("7d7d7d", 'GUM', (125, 125, 125)),
}

In [ ]:
def project_point(rt, k, point):
    # 将点P的三维坐标转换为齐次坐标形式
    point_homogeneous = np.array([point[0], point[1], point[2], 1])

    # 将点P的三维坐标转换到相机坐标系中
    point_cam = np.dot(rt, point_homogeneous)
    point_cam = point_cam[:3]  # 忽略最后一个值1

    # 将点P在相机坐标系中的坐标转换到图像平面坐标系中
    point_img = np.dot(k, point_cam)

    # 将图像平面坐标系中的坐标转换为二维坐标形式
    img_x = point_img[0] / point_img[2]
    img_y = point_img[1] / point_img[2]

    return img_x, img_y

In [ ]:
def show_ply(vertices, colors, save_path):
    header = (f"ply\n"
              f"format ascii 1.0\n"
              f"comment VCGLIB generated\n"
              f"element vertex {vertices.shape[0]}\n"
              f"property double x\n"
              f"property double y\n"
              f"property double z\n"
              f"property uchar red\n"
              f"property uchar green\n"
              f"property uchar blue\n"
              f"property uchar alpha\n"
              f"element face {0}\n"
              f"property list uchar int vertex_indices\n"
              f"property uchar red\n"
              f"property uchar green\n"
              f"property uchar blue\n"
              f"property uchar alpha\n"
              f"end_header\n")

    vertex_info = ""
    for color, coord in zip(colors, vertices):
        vertex_info += f'{coord[0]} {coord[1]} {coord[2]} {color[0]} {color[1]} {color[2]} {255}\n'

    with open(save_path, 'w', encoding='ascii') as f:
        f.write(header)
        f.write(vertex_info)


def m3dLookAt(eye, target, up):
    def normalize(v):
        return v / np.sqrt(np.sum(v ** 2))

    mz = normalize(eye - target)
    mx = normalize(np.cross(up, mz))
    my = normalize(np.cross(mz, mx))

    return np.array([
        [mx[0], my[0], mz[0], eye[0]],
        [mx[1], my[1], mz[1], eye[1]],
        [mx[2], my[2], mz[2], eye[2]],
        [0, 0, 0, 1]
    ])


# 判断点是否在三角形内
def is_point_in_triangle(px, py, x1, y1, x2, y2, x3, y3):
    def area(x1, y1, x2, y2, x3, y3):
        return abs(x1 * (y2 - y3) + x2 * (y3 - y1) + x3 * (y1 - y2))

    total_area = area(x1, y1, x2, y2, x3, y3)
    area1 = area(px, py, x2, y2, x3, y3)
    area2 = area(x1, y1, px, py, x3, y3)
    area3 = area(x1, y1, x2, y2, px, py)

    return abs(total_area - (area1 + area2 + area3)) < 1e-3


# 获取三角形覆盖的坐标
def get_covered_points(x1, y1, x2, y2, x3, y3):
    min_x = min(x1, x2, x3)
    max_x = max(x1, x2, x3)
    min_y = min(y1, y2, y3)
    max_y = max(y1, y2, y3)

    covered_points = []

    for x in range(min_x, max_x + 1):
        for y in range(min_y, max_y + 1):
            if is_point_in_triangle(x, y, x1, y1, x2, y2, x3, y3):
                covered_points.append((x, y))

    return covered_points

In [ ]:
def render(label_path, save_path, rend_size=(256, 256), rend_step=(6, 9)): 
    """
    rend_step : (beta, theta)
    """
    print(label_path)
    base_name = os.path.basename(label_path)[:-4]

    os.makedirs(os.path.join(save_path, base_name), exist_ok=True)
    os.makedirs(os.path.join(save_path, base_name, 'img'), exist_ok=True)
    os.makedirs(os.path.join(save_path, base_name, 'label'), exist_ok=True)
    os.makedirs(os.path.join(save_path, base_name, 'p2p'), exist_ok=True)

    label_trimesh = trimesh.load(label_path)
    vertices = np.asarray(label_trimesh.vertices)
    num_cells = len(label_trimesh.faces)
    minCoord = np.min(vertices, axis=0)
    maxCoord = np.max(vertices, axis=0)
    meanCoord = np.mean(vertices, axis=0)
    x_len, y_len, z_len = maxCoord - minCoord
    # 计算球体半径
    radius = np.sqrt(np.sum((maxCoord - meanCoord) ** 2)) * 1.2
    theta = math.asin((z_len / 2) / radius)

    # 记录所有的相机位姿
    camera_pos_list = []

    while theta <= math.pi / 2:
        z_offset = math.sin(theta) * radius
        sub_radius = math.cos(theta) * radius
        beta = 0
        while beta <= math.pi * 2:
            x_offset = sub_radius * math.cos(beta)
            y_offset = sub_radius * math.sin(beta)
            camera_pos = meanCoord + np.asarray([x_offset, y_offset, z_offset], dtype=float)

            # 计算一个上方向
            camera_pos_list.append((
                m3dLookAt(camera_pos, meanCoord, np.asarray([0, 0, 1], dtype=float)),
                theta,
                beta,
                camera_pos
            ))
            beta += math.pi / rend_step[0]
        theta += math.pi / rend_step[1]

    # 创建模型
    pyrender_mesh = pyrender.Mesh.from_trimesh(label_trimesh)
    

    # 创建场景
    scene = pyrender.Scene()
    label_scene = pyrender.Scene()

    # 场景添加模型
    scene.add(pyrender_mesh)
    # 标签实例分割需要获取单独实例
    seg_node_map = {}
    vertex_instances = {}
    label_color_map = {}
    face_instances = {}
    # 获取顶点颜色信息
    for i, vertex_color in enumerate(label_trimesh.visual.vertex_colors):
        vertex_color = (vertex_color[0], vertex_color[1], vertex_color[2])
        if vertex_color in color2label:
            vertex_label = color2label[vertex_color][2]
            if not vertex_label in vertex_instances:
                vertex_instances[vertex_label] = {}
            vertex_instances[vertex_label][i] = len(vertex_instances[vertex_label])
            label_color_map[vertex_label] = vertex_color

    # 获取面片颜色信息
    if 'face' in label_trimesh.metadata['_ply_raw']:
        face_meta = label_trimesh.metadata['_ply_raw']['face']
        if 'red' in face_meta['data'] and 'green' in face_meta['data'] and 'blue' in face_meta['data']:
            face_colors = np.stack([
                face_meta['data']['red'],
                face_meta['data']['green'],
                face_meta['data']['blue']
            ], axis=-1).squeeze(1)  # shape: (N_faces, 3)

    for i, face in enumerate(label_trimesh.faces):
        face_color = tuple(face_colors[i])  # 取RGB，忽略alpha

        if face_color in color2label:
            label = color2label[face_color][2]

            if label not in face_instances:
                face_instances[label] = []

            # 注意：此时 face 是原始 mesh 的顶点索引，我们稍后再映射到局部索引
            face_instances[label].append(face)
    for label, faces in face_instances.items():
        # 收集所有该 label 用到的顶点索引
        vertex_indices = set([vid for f in faces for vid in f])

        # 构建原始顶点到局部顶点的映射
        vertex_idx_map = {v: i for i, v in enumerate(vertex_indices)}

        # 构建局部顶点和三角形索引
        vertice_node = np.array([label_trimesh.vertices[v] for v in vertex_indices])
        face_node = np.array([[vertex_idx_map[v] for v in f] for f in faces])

        label_color = label_color_map[label]
        vertice_color_node = np.array([label_color] * vertice_node.shape[0])
        face_color_node = np.array([label_color] * face_node.shape[0])
        mesh_node = trimesh.Trimesh(vertices=vertice_node, faces=face_node, vertex_colors=vertice_color_node, face_colors=face_color_node)
        # 当前模型添加到场景中
        node = label_scene.add(pyrender.Mesh.from_trimesh(mesh_node))
        seg_node_map[node] = label_color


    # 添加光源
    light = pyrender.DirectionalLight(color=[1.0, 1.0, 1.0], intensity=1.0)
    scene.add(light)
    label_scene.add(light)

    # 创建相机
    camera = pyrender.PerspectiveCamera(yfov=np.pi / 2, aspectRatio=1.0)

    # 渲染参数
    r = pyrender.OffscreenRenderer(viewport_width=rend_size[1],
                                   viewport_height=rend_size[0],
                                   point_size=1.0)


    # 每个面片的标签，按照不同视角下的像素标签投票
    cell_label_vote = np.zeros((num_cells, 17))
    camera_params = []

    for i, camera_pos in enumerate(tqdm(camera_pos_list, desc="Processing camera positions")):

        # 渲染图片
        camera_node = scene.add(camera, pose=camera_pos[0])
        img_color, img_depth = r.render(scene)
        scene.remove_node(camera_node)
        plt.imsave(os.path.join(save_path, base_name, 'img', f'{base_name}_{i}.png'), img_color)

        # 渲染分割标签
        camera_node = label_scene.add(camera, pose=camera_pos[0])
        label_color, label_depth = r.render(label_scene, flags=pyrender.RenderFlags.SEG, seg_node_map=seg_node_map)
        label_scene.remove_node(camera_node)
        plt.imsave(os.path.join(save_path, base_name, 'label', f'{base_name}_{i}.png'), label_color)


        # 口扫点云坐标对应到图片像素坐标
        point_coords = np.asarray(label_trimesh.vertices)
        cells = np.asarray(label_trimesh.faces)
        # 面片重心点坐标
        cell_coords = np.array([[
            (point_coords[point_idxs[0]][0] + point_coords[point_idxs[1]][0] + point_coords[point_idxs[2]][0]) / 3,
            (point_coords[point_idxs[0]][1] + point_coords[point_idxs[1]][1] + point_coords[point_idxs[2]][1]) / 3,
            (point_coords[point_idxs[0]][2] + point_coords[point_idxs[1]][2] + point_coords[point_idxs[2]][2]) / 3,
        ] for point_idxs in cells])
        # 面片到相机的距离
        cell_distances = np.sqrt(np.sum((cell_coords - camera_pos[3]) ** 2, axis=1))
        # 用面片到相机的距离将面片索引按从近到远排序
        sorted_cell_indices = np.argsort(cell_distances)

        # 点云坐标到图像坐标的映射 (H, W, 2) 2: cell index, camera distance
        pixel_point_map = np.asarray([[[-1, 0]] * rend_size[0]] * rend_size[1])

        # 获取相机参数
        Rt = np.eye(4)
        Rt[:3, :3] = camera_pos[0][:3, :3].T
        Rt[:3, 3] = -np.dot(camera_pos[0][:3, :3].T, camera_pos[0][:3, 3])
        # 焦距f
        f_y = (rend_size[0] / 2) / math.tan(np.pi / 2 / 2)
        f_x = f_y * 1.0
        # 光心c
        cx, cy = rend_size[1] / 2.0, rend_size[0] / 2.0
        K = np.array(
            [[f_x, 0, cx],
                [0, f_y, cy],
                [0, 0, 1]]
        )
        param_dict = {
            "frame": i,
            "theta": float(camera_pos[1]),
            "beta": float(camera_pos[2]),
            "Rt": Rt.tolist(),
            "K": K.tolist()
        }
        camera_params.append(param_dict)

        for cell_idx in sorted_cell_indices:
            # 面片到相机距离
            distance = cell_distances[cell_idx]

            # 面片中三个顶点在图像上的坐标
            cell_pixel_point = project_point(Rt, K, cell_coords[cell_idx])
            cell_pixel_point = [cell_pixel_point[1], rend_size[0] - cell_pixel_point[0]]
            cell_pixel_point = [round(cell_pixel_point[0]), round(cell_pixel_point[1])]

            # 超出像素坐标，跳过
            if cell_pixel_point[0] < 0 or cell_pixel_point[0] >= rend_size[0] or cell_pixel_point[1] < 0 or cell_pixel_point[1] >= rend_size[1]:
                continue

            pixel_point_0 = project_point(Rt, K, point_coords[cells[cell_idx][0]])
            pixel_point_0 = [pixel_point_0[1], rend_size[0] - pixel_point_0[0]]
            pixel_point_0 = [round(pixel_point_0[0]), round(pixel_point_0[1])]

            # 超出像素坐标，跳过
            if pixel_point_0[0] < 0 or pixel_point_0[0] >= rend_size[0] or pixel_point_0[1] < 0 or pixel_point_0[1] >= rend_size[1]:
                continue

            pixel_point_1 = project_point(Rt, K, point_coords[cells[cell_idx][1]])
            pixel_point_1 = [pixel_point_1[1], rend_size[0] - pixel_point_1[0]]
            pixel_point_1 = [round(pixel_point_1[0]), round(pixel_point_1[1])]

            # 超出像素坐标，跳过
            if pixel_point_1[0] < 0 or pixel_point_1[0] >= rend_size[0] or pixel_point_1[1] < 0 or pixel_point_1[1] >= rend_size[1]:
                continue

            pixel_point_2 = project_point(Rt, K, point_coords[cells[cell_idx][2]])
            pixel_point_2 = [pixel_point_2[1], rend_size[0] - pixel_point_2[0]]
            pixel_point_2 = [round(pixel_point_2[0]), round(pixel_point_2[1])]

            # 超出像素坐标，跳过
            if pixel_point_2[0] < 0 or pixel_point_2[0] >= rend_size[0] or pixel_point_2[1] < 0 or pixel_point_2[1] >= rend_size[1]:
                continue

            # 三个顶点组成的面片，占据的图片像素点的离散整数坐标
            covered_coords = get_covered_points(pixel_point_0[0], pixel_point_0[1], pixel_point_1[0], pixel_point_1[1], pixel_point_2[0], pixel_point_2[1])
            covered_coords.append((cell_pixel_point[0], cell_pixel_point[1]))

            # 一个cell可能会覆盖多个像素点，包含多个颜色信息
            for pixel_point in covered_coords:
                if pixel_point_map[pixel_point[0], pixel_point[1]][0] < 0:
                    pixel_point_map[pixel_point[0], pixel_point[1]] = np.asarray([cell_idx, distance])
                elif pixel_point_map[pixel_point[0], pixel_point[1]][1] > distance:
                    pixel_point_map[pixel_point[0], pixel_point[1]] = np.asarray([cell_idx, distance])

        # 使用render mask的颜色给原始的3d mesh上色
        color = img_color.copy() # 基于render image的颜色，用render mask的颜色覆盖
        map_color = np.asarray([[125, 125, 125]] * cell_coords.shape[0]) # 表示每个面片的颜色，从render mask中获取
        for x in range(rend_size[0]):
            for y in range(rend_size[1]):
                if pixel_point_map[x, y][0] > 0:
                    # 找到对应的图像标签颜色
                    point_color = pixel_color = label_color[x, y]
                    # 标签黑色表示不是前景，使用原始渲染图片的颜色
                    if pixel_color[0] == 0 and pixel_color[1] == 0 and pixel_color[2] == 0:
                        pixel_color = img_color[x, y]
                        point_color = np.asarray([255, 255, 255])
                    map_color[pixel_point_map[x, y][0]] = point_color
                    color[x, y] = pixel_color
                    point_color = (point_color[0], point_color[1], point_color[2])
                    cell_label_vote[pixel_point_map[x, y][0], color2label[point_color][2] if point_color in color2label else 0] += 1

        # 可视化点云对应关系，在每个不同的视角下
        show_ply(cell_coords, map_color, os.path.join(save_path, base_name, 'p2p', f'{base_name}_{i}.ply'))

        # 可视化图片对应关系
        plt.imsave(os.path.join(save_path, base_name, 'p2p', f'{base_name}_{i}.png'), color)

    # 综合所有视角的颜色，点云标签选择投票最多的，可视化输出
    cell_label_vote = np.argmax(cell_label_vote, axis=1)
    if 'lower' in base_name:
        cell_color_vote = np.asarray([label2color_lower[label][2] if label > 0 else (255, 255, 255) for label in cell_label_vote])
    elif 'upper' in base_name:
        cell_color_vote = np.asarray([label2color_upper[label][2] if label > 0 else (255, 255, 255) for label in cell_label_vote])
    show_ply(cell_coords, cell_color_vote, os.path.join(save_path, base_name, f'{base_name}_vote.ply'))

    # 保存相机信息
    with open(os.path.join(os.path.join(save_path, base_name, f'{base_name}_view.json')), 'w') as f:
        json.dump(camera_params, f, indent=4)

    r.delete()



In [ ]:
render(
    'tmp/YBSESUN6_upper_gt.ply',
    'tmp',
    rend_size=(1024, 1024),
    rend_step=(2, 4)
)